# Fuzzy Grouping
Copyright (c) Microsoft Corporation. All rights reserved.<br>
Licensed under the MIT License.<br>

Unprepared data often represents the same entity with multiple values; examples include different spellings, varying capitalizations, and abbreviations. This is common when working with data gathered from multiple sources or through human input. One way to canonicalize and reconcile these variants is to use Data Prep's fuzzy_group_column (also known as "text clustering") functionality.

Data Prep inspects a column to determine clusters of similar values. A new column is added in which clustered values are replaced with the canonical value of its cluster, thus significantly reducing the number of distinct values. You can control the degree of similarity required for values to be clustered together, override canonical form, and set clusters if automatic clustering did not provide the desired results.

Let's explore the capabilities of `fuzzy_group_column` by first reading in a dataset and inspecting it.

In [1]:
import azureml.dataprep as dprep

In [2]:
dflow = dprep.read_json(path='../data/json.json')
dflow.head(5)

inspections.business.business_id  inspections.business.name  \
0                            16162  Quick-N-Ezee Indian Foods   
1                            69707       Little Green Cyclo 2   
2                            67565  King of Thai Noodles Cafe   
3                            67565  King of Thai Noodles Cafe   
4                            68701                     Grindz   

  inspections.business.address inspections.business.city  \
0                3861 24th St                         SF   
1               Off The Grid                               
2             1541 TARAVAL St              SAN FRANCISCO   
3             1541 TARAVAL St              SAN FRANCISCO   
4              832 Clement St                         SF   

  inspections.business.postal_code inspections.business.latitude  \
0                            94114                                 
1                                                                  
2                            94116                       37.7427   
3                            94116                       37.7427   
4                            94118                       37.7828   

  inspections.business.longitude inspections.business.phone_number  \
0                                                                    
1                                                                    
2                       -122.483                                     
3                       -122.483                                     
4                       -122.468                                     

  inspections.business.TaxCode inspections.business.business_certificate  \
0                          H34                                    467114   
1                          H79                                    453248   
2                          H25                                             
3                          H25                                             
4                          H25                                    467498   

  inspections.business.application_date inspections.business.owner_name  \
0                   May  9 2005 12:00AM            Jagpreet Enterprises   
1                   Jul 12 2012 12:00AM            LITTLEGREENCYCLO LLC   
2                   Oct 12 2011 12:00AM         Royal Thai Noodles, Inc   
3                   Oct 12 2011 12:00AM         Royal Thai Noodles, Inc   
4                   Mar 16 2012 12:00AM                 Ono Grindz, LLC   

                  inspections.business.owner_address inspections.Score  \
0         23682 Clawiter Road\n Hayward\n CA\n 94545               100   
1  100 Esplanade Ave., Apt. 99\n Pacifica\n CA\n ...                93   
2                    2410 19th Ave\n SF\n CA\n 94116                79   
3                    2410 19th Ave\n SF\n CA\n 94116                     
4            1055 Granada St.\n Vallejo\n CA\n 94591               100   

  inspections.date       inspections.type  \
0         20130223  Routine - Unscheduled   
1         20130224  Routine - Unscheduled   
2         20130225  Routine - Unscheduled   
3         20130225              Complaint   
4         20130225  Routine - Unscheduled   

                              inspections.violations  
0                                                 []  
1  [{"description":"103112: No hot water or runni...  
2  [{"description":"103139: Improper food storage...  
3  [{"description":"103139: Improper food storage...  
4                                                 []

As you can see above, the column `inspections.business.city` contains several forms of the city name "San Francisco".
Let's add a column with values replaced by the automatically detected canonical form. To do so call fuzzy_group_column() on an existing Dataflow:

In [3]:
dflow_clean = dflow.fuzzy_group_column(source_column='inspections.business.city',
                                       new_column_name='city_grouped',
                                       similarity_threshold=0.8,
                                       similarity_score_column_name='similarity_score')
dflow_clean.head(5)

inspections.business.business_id  inspections.business.name  \
0                            16162  Quick-N-Ezee Indian Foods   
1                            69707       Little Green Cyclo 2   
2                            67565  King of Thai Noodles Cafe   
3                            67565  King of Thai Noodles Cafe   
4                            68701                     Grindz   

  inspections.business.address inspections.business.city   city_grouped  \
0                3861 24th St                         SF  San Francisco   
1               Off The Grid                                              
2             1541 TARAVAL St              SAN FRANCISCO  San Francisco   
3             1541 TARAVAL St              SAN FRANCISCO  San Francisco   
4              832 Clement St                         SF  San Francisco   

   similarity_score inspections.business.postal_code  \
0          0.814806                            94114   
1               NaN                                    
2          1.000000                            94116   
3          1.000000                            94116   
4          0.814806                            94118   

  inspections.business.latitude inspections.business.longitude  \
0                                                                
1                                                                
2                       37.7427                       -122.483   
3                       37.7427                       -122.483   
4                       37.7828                       -122.468   

  inspections.business.phone_number inspections.business.TaxCode  \
0                                                            H34   
1                                                            H79   
2                                                            H25   
3                                                            H25   
4                                                            H25   

  inspections.business.business_certificate  \
0                                    467114   
1                                    453248   
2                                             
3                                             
4                                    467498   

  inspections.business.application_date inspections.business.owner_name  \
0                   May  9 2005 12:00AM            Jagpreet Enterprises   
1                   Jul 12 2012 12:00AM            LITTLEGREENCYCLO LLC   
2                   Oct 12 2011 12:00AM         Royal Thai Noodles, Inc   
3                   Oct 12 2011 12:00AM         Royal Thai Noodles, Inc   
4                   Mar 16 2012 12:00AM                 Ono Grindz, LLC   

                  inspections.business.owner_address inspections.Score  \
0         23682 Clawiter Road\n Hayward\n CA\n 94545               100   
1  100 Esplanade Ave., Apt. 99\n Pacifica\n CA\n ...                93   
2                    2410 19th Ave\n SF\n CA\n 94116                79   
3                    2410 19th Ave\n SF\n CA\n 94116                     
4            1055 Granada St.\n Vallejo\n CA\n 94591               100   

  inspections.date       inspections.type  \
0         20130223  Routine - Unscheduled   
1         20130224  Routine - Unscheduled   
2         20130225  Routine - Unscheduled   
3         20130225              Complaint   
4         20130225  Routine - Unscheduled   

                              inspections.violations  
0                                                 []  
1  [{"description":"103112: No hot water or runni...  
2  [{"description":"103139: Improper food storage...  
3  [{"description":"103139: Improper food storage...  
4                                                 []

The arguments `source_column` and `new_column_name` are required, whereas the others are optional.
If `similarity_threshold` is provided, it will be used to control the required similarity level for the values to be grouped together.
If `similarity_score_column_name` is provided, a second new column will be added to show similarity score between every pair of original and canonical values.

In the resulting data set, you can see that all the different variations of representing "San Francisco" in the data were normalized to the same string, "San Francisco".

But what if you want more control over what gets grouped, what doesn't, and what the canonical value should be? 

To get more control over grouping, canonical values, and exceptions, you need to use the `FuzzyGroupBuilder` class.
Let's see what it has to offer below:

In [4]:
builder = dflow.builders.fuzzy_group_column(source_column='inspections.business.city',
                                            new_column_name='city_grouped',
                                            similarity_threshold=0.8,
                                            similarity_score_column_name='similarity_score')

In [5]:
# calling learn() to get fuzzy groups
builder.learn()
builder.groups

[{'canonicalValue': 'San Francisco',
  'duplicates': [{'duplicateValue': 'San Francisco',
    'similarityScore': 1.0,
    'useForReplacement': True},
   {'duplicateValue': 'SAN FRANCISCO',
    'similarityScore': 1.0,
    'useForReplacement': True},
   {'duplicateValue': 'SF',
    'similarityScore': 0.8148061037063599,
    'useForReplacement': True},
   {'duplicateValue': 'S.F.',
    'similarityScore': 0.8148061037063599,
    'useForReplacement': True}]}]

Here you can see that `fuzzy_group_column` detected one group with four values that all map to "San Francisco" as the canonical value.
You can see the effects of changing the similarity threshold next:

In [6]:
builder.similarity_threshold = 0.9
builder.learn()
builder.groups

[{'canonicalValue': 'SF',
  'duplicates': [{'duplicateValue': 'SF',
    'similarityScore': 1.0,
    'useForReplacement': True},
   {'duplicateValue': 'S.F.',
    'similarityScore': 0.9523809552192688,
    'useForReplacement': True}]},
 {'canonicalValue': 'San Francisco',
  'duplicates': [{'duplicateValue': 'San Francisco',
    'similarityScore': 1.0,
    'useForReplacement': True},
   {'duplicateValue': 'SAN FRANCISCO',
    'similarityScore': 1.0,
    'useForReplacement': True}]}]

Now that you are using a similarity threshold of `0.9`, two distinct groups of values are generated.

Let's tweak some of the detected groups before completing the builder and getting back the Dataflow with the resulting fuzzy grouped column.

In [7]:
builder.similarity_threshold = 0.8
builder.learn()
groups = builder.groups
groups

[{'canonicalValue': 'San Francisco',
  'duplicates': [{'duplicateValue': 'San Francisco',
    'similarityScore': 1.0,
    'useForReplacement': True},
   {'duplicateValue': 'SAN FRANCISCO',
    'similarityScore': 1.0,
    'useForReplacement': True},
   {'duplicateValue': 'SF',
    'similarityScore': 0.8148061037063599,
    'useForReplacement': True},
   {'duplicateValue': 'S.F.',
    'similarityScore': 0.8148061037063599,
    'useForReplacement': True}]}]

In [8]:
# change the canonical value for the first group
groups[0]['canonicalValue'] = 'SANFRAN'
duplicates = groups[0]['duplicates']
# remove the last duplicate value from the cluster
duplicates = duplicates[:-1]
# assign modified duplicate array back
groups[0]['duplicates'] = duplicates
# assign modified groups back to builder
builder.groups = groups
builder.groups

[{'canonicalValue': 'SANFRAN',
  'duplicates': [{'duplicateValue': 'San Francisco',
    'similarityScore': 1.0,
    'useForReplacement': True},
   {'duplicateValue': 'SAN FRANCISCO',
    'similarityScore': 1.0,
    'useForReplacement': True},
   {'duplicateValue': 'SF',
    'similarityScore': 0.8148061037063599,
    'useForReplacement': True}]}]

Here, the canonical value is modified to be used for the single fuzzy group and removed 'S.F.' from this group's duplicates list.

You can mutate the copy of the `groups` list from the builder (be careful to keep the structure of objects inside this list). After getting the desired groups in the list, you can update the builder with it.

Now you can get a dataflow with the FuzzyGroup step in it.

In [9]:
dflow_clean = builder.to_dataflow()

df = dflow_clean.to_pandas_dataframe()
df

inspections.business.business_id        inspections.business.name  \
0                             16162        Quick-N-Ezee Indian Foods   
1                             69707             Little Green Cyclo 2   
2                             67565        King of Thai Noodles Cafe   
3                             67565        King of Thai Noodles Cafe   
4                             68701                           Grindz   
5                             69186  Premier Catering & Events, Inc.   
6                              2689                   THE BLUE PLATE   
7                             15806                   Vital Tea Leaf   
8                             21807                  The Front Porch   
9                             69041                  Washington Cafe   
10                             6018      3RD BAPTIST FELLOWSHIP HALL   
11                            23702                Doo Bu Restaurant   
12                            23702                Doo Bu Restaurant   
13                            34244               Vital Tea Leaf Inc   
14                            63503     Mason Street Deli and Market   
15                             1427                       MIFUNE DON   
16                             3331                         HARVEY'S   
17                             5907     LILIENTHAL ELEMENTARY SCHOOL   
18                            65050                 House of Banquet   
19                            71534            7 Eleven #2366-36039A   
20                            71769              Lucky One Mini Mart   
21                             5884   Gordon J Lau Elementary School   
22                            69635                  Sophie's Crepes   
23                             3633           Magnolia Pub & Brewery   
24                             5905             MARINA MIDDLE SCHOOL   
25                            19277                     Farmer Brown   
26                             1775                       Sapporo-Ya   
27                            61135  Old Mandarin Islamic Restaurant   
28                              765                APERTO RESTAURANT   
29                             3373             New Delhi Restaurant   
30                             3373             New Delhi Restaurant   
31                            33891                           Harlot   
32                            28828                The Sandwich Shop   
33                            60933                            Mayes   
34                            62688                   Auntie April's   
35                             3373             New Delhi Restaurant   
36                             2756                        STARBUCKS   
37                             7489                       GAS & SHOP   
38                            19094                           Rigolo   
39                            65051                   Mission Cheese   
40                             3802    THE GREAT AMERICAN MUSIC HALL   
41                              475                           Mara's   
42                             1193            PEET'S COFFEES & TEAS   
43                             2548     NOAH'S NEW YORK BAGELS #2120   
44                             6913                    XTRA OIL CORP   
45                             4923                     TED'S MARKET   
46                            36020                  Cadillac Market   
47                             3935             Le Colonial, SF, LLC   
48                             1259              KD'S GROG & GROCERY   
49                            65396                 Carmelina's Cafe   
50                              477                       Il Pollaio   
51                             1199                         PANCHO'S   
52                             1549                     Cafe Bellini   
53                             4257                           DELUXE   
54                            68775        